In [2]:
import numpy as np
import pandas as pd
import os, random
from tqdm import tqdm # 진행도 시각화를 위한 라이브러리

seed=42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)

## 데이터 불러오기

In [3]:
train_data = pd.read_csv("nsmc_train.csv", index_col=0)
test_data = pd.read_csv("nsmc_test.csv", index_col=0)
print(train_data.shape)
print(test_data.shape)

(149993, 2)
(49999, 1)


In [4]:
train_data.head()

,review,rating
id,,
9324809,배우들의 인생연기가 돋보였던... 최고의 드라마,1
9305425,아 혜리 보고싶다 ... 여군좀 ㅠ,0
5239110,"눈이 팅팅..... 정말 ,..... 대박이다......",1
9148159,캐슬린 터너의 보디는 볼만했다,0
6144938,진짜 최고였다.,1


In [5]:
x_train = train_data["review"]
y_train = np.array(train_data["rating"])

## 전처리


- 불용어 제거

In [6]:
import re

pattern = r"[^\uAC00-\uD7A3a-zA-Z\s]"

def apply_regex(pattern, text):  # 정규표현식을 이용한 필터링 적용
    text = re.sub(pattern, "", text)  # 정규표현식 패턴에 맞는 값들을 텍스트에서 제거
    text = text.upper() # 영어들을 찾아 대문자로 치환하는 코드 작성
    return text

x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]

C:\Users\VISLAB-seongwon\AppData\Local\Temp\ipykernel_22616\3170241328.py:10: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  x_train_preprocessed = [apply_regex(pattern, str(x[1])) for x in tqdm(x_train.iteritems(), total=len(x_train), desc="pre-processing data")]
pre-processing data: 100%|██████████| 149993/149993 [00:00<00:00, 390804.22it/s]


- 토큰화

In [7]:
from konlpy.tag import Okt
okt = Okt()

def tokenize_words(sentence):
    sentence_n = okt.normalize(sentence)
    sentence_tokenized = okt.morphs(sentence_n, stem=True)
    #sentence_tokenized = okt.morphs(sentence_n, norm=True, stem=True)
    return sentence_tokenized

In [8]:
# 약 10-15분 정도 소요됩니다. 
x_train_tokenized = [tokenize_words(x) for x in tqdm(x_train_preprocessed, desc="tokenizing data")]

tokenizing data:  16%|█▋        | 24606/149993 [01:00<04:42, 443.92it/s]

- 불용어 제거

In [ ]:
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']  #별다른 의미가 없는 불용어들

def exclude_stopwords(text):
    # 위 리스트에 포함된 불용어들을 제거하는 코드 작성
    text_n = [word for word in text if word not in stopwords]
    return text_n

x_train_stopwords_excluded = [exclude_stopwords(x) for x in x_train_tokenized]

In [ ]:
len(x_train_stopwords_excluded)